In [1]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

Defaulting to user installation because normal site-packages is not writeable


In [19]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "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"

In [3]:
! pip install langchain-ollama

Defaulting to user installation because normal site-packages is not writeable


In [4]:
! pip install pypdf

Defaulting to user installation because normal site-packages is not writeable


In [20]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate

In [21]:
#### INDEXING ####

# Load Documents
file_path = "/Users/kryptonempyrean/Desktop/TS3043166.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

Ignoring wrong pointing object 169 0 (offset 0)
Ignoring wrong pointing object 178 0 (offset 0)


In [22]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [23]:
# Embeddings (use HuggingFace for local embedding)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [24]:
# Vector Store
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [25]:
#### RETRIEVAL + GENERATION ####

# Prompt 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Use the following context to answer the question. Be concise and specific.

Context:
{context}

Question:
{question}
""",
)

In [26]:
# LLM using Ollama and LLaMA 3
llm = Ollama(model="llama3", temperature=0)

In [27]:
# Format function
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [28]:
# RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
# Run the chain
response = rag_chain.invoke("What is CLALIT?")
print(response)
print(retriever)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


CLALIT is the largest provider of public and semi-private health services in Israel, which runs as a not-for-profit entity under Israeli law.
tags=['Chroma', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x13d06d280> search_kwargs={}


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [29]:
# Step 1: Retrieve relevant docs
query = "What is CLALIT?"
retrieved_docs = retriever.invoke(query)

# Step 2: Format the context
context = format_docs(retrieved_docs)

# Optional: Print the context
print("📚 CONTEXT:\n")
print(context)
print("\n" + "="*80 + "\n")

# Step 3: Run prompt → LLM → Output parser
filled_prompt = prompt.format(context=context, question=query)
response = llm.invoke(filled_prompt)
response = StrOutputParser().invoke(response)

# Step 4: Print the final answer
print("🤖 RESPONSE:\n")
print(response)


📚 CONTEXT:

2D a t a a n d M e t h o d s
2.1 Data Description
The following work is based on the analysis of a dataset retrieved from the
CLALIT Health Services Israeli database. This is the largest provider of
public and semi-private health services in Israel. Under Israeli law, it is run
as a not-for-proﬁt entity. The data at hand has been collected between Jan-
uary, 2002 and December 2022. It has been optimally designed to study the
relation between Psoriasis and Male Infertility in the available subjects. Full
anonimity of subjects has been assured by the Services of Israel, ensuring all
patient information was de-identiﬁed.
The CLALIT dataset, maintained by Israel’s largest health services organi-
zation, provides comprehensive records for randomly selected members who
receive routine healthcare services. All medical diagnoses (including psori-
asis and various comorbidities) are systematically entered by the treating

2D a t a a n d M e t h o d s
2.1 Data Description
The followi

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [30]:
# Step 1: Retrieve relevant docs
query = "Hoe does Arthropathy relate to male infertility?"
retrieved_docs = retriever.invoke(query)

# Step 2: Format the context
context = format_docs(retrieved_docs)

# Optional: Print the context
print("📚 CONTEXT:\n")
print(context)
print("\n" + "="*80 + "\n")

# Step 3: Run prompt → LLM → Output parser
filled_prompt = prompt.format(context=context, question=query)
response = llm.invoke(filled_prompt)
response = StrOutputParser().invoke(response)

# Step 4: Print the final answer
print("🤖 RESPONSE:\n")
print(response)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


📚 CONTEXT:

anisms and novel treatment approaches. Furthermore, recent biological and
clinical studies have uncovered a subtle but deep relation with conditions of
male infertility in diﬀerent psoriatic patients. This is precisely, the point that
this study tries to focus on.
According to the World Health Organization (WHO), infertility is deﬁned
as the absence of conception after 12–24 months of regular, unprotected in-
tercourse. However, the term “infertility” can be reﬁned into true “sterility,”
where no medical intervention can restore fertility, and “subfertility,” which
may be overcome with appropriate treatments. About 15–20% of couples
experience diﬃculty conceiving, inﬂuenced partly by social factors (e.g., de-
layed family planning).
Male infertility can be classiﬁed as pre-testicular (secretory) or testicular
9

anisms and novel treatment approaches. Furthermore, recent biological and
clinical studies have uncovered a subtle but deep relation with conditions of
male inferti

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


🤖 RESPONSE:

According to the context, there is a subtle but deep relation between conditions of psoriatic arthropathy and male infertility. The study aims to focus on this point, suggesting that there may be an underlying connection between the two.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
